In [ ]:
!pip  install "langchain==0.1.10" "ibm-watsonx-ai>=0.2.6" langchain-ibm==0.1.5 langchain-community==0.0.25 langchain-core>=0.1.28 sentence-transformers "pydantic==1.10.11" "sqlalchemy==2.0.1"
!pip  install  chromadb==0.4.4
!pip install pysqlite3-binary wget

In [4]:
# Add API key and project_id
import os, getpass
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

In [ ]:
# RAG Document data loading

import wget
filename = 'jesc109.txt'
url = 'https://raw.githubusercontent.com/prabhav-thali/quality-education/refs/heads/main/SourceCode/RAG_implementation/jesc109.txt'

if not os.path.isfile(filename):
    wget.download(url, out=filename)

In [ ]:
# Build up knowledge base

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(f"Number of splits:{len(texts)}")	
	

In [ ]:
# embedding function

from ibm_watsonx_ai.foundation_models.utils import get_embedding_model_specs
get_embedding_model_specs(credentials.get('url'))
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id
    )
docsearch = Chroma.from_documents(texts, embeddings)

In [ ]:
# Foundation Models on watsonx.ai
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
model_id = ModelTypes.LLAMA_3_70B_INSTRUCT
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}


# LangChain CustomLLM wrapper for watsonx model
from langchain_ibm import WatsonxLLM

watsonx_granite = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

In [ ]:
# Generate a retrieval-augmented response to a question

from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=watsonx_granite, chain_type="stuff", retriever=docsearch.as_retriever())
query = "What are the laws of reflection of light?"
qa.invoke(query)